In [1]:
# this is just a temp fix to get things working locally. use the server config when starting jupyter
import sys, os

# NOTE: change this path if you need to for your system
client_side_path = os.getcwd()
print("client_side_path: %s" % (client_side_path))
sys.path.append(client_side_path)

client_side_path: /media/dpihera/Extra Drive 1/development/jupyternb_c_app_websockets/client_side


In [2]:
from wsclient.client import TestWebSocketClient
from tornado import ioloop

print("IOLoop: %s" % (ioloop.IOLoop.current()))
client = TestWebSocketClient()
client.connect('ws://localhost:8675/ws')

IOLoop: <zmq.eventloop.ioloop.ZMQIOLoop object at 0x7ff90fbab9b0>
WebSocketClient.__init__: cto, rto = 10, 60
WebSocketClient.connect: attempting connection to: ws://localhost:8675/ws
WebSocketClient.connect: ws_conn is: <tornado.concurrent.Future object at 0x7ff918db5a90>
WebSocketClient.connect: adding to ioloop (<zmq.eventloop.ioloop.ZMQIOLoop object at 0x7ff90fbab9b0>)
WebSocketClient.connect: done


Connected to Websocket Server!
client_side.TestWebSocketClient._on_message: precessing Hello World from the server_side


In [3]:
d = {'does_it': 'blend'}
client.send(d)

client_side.TestWebSocketClient._on_message: precessing {does_it, blend} from the server_side
